In [1]:
import geopandas
import shapefile
import pandas as pd
import json

In [2]:
reader = shapefile.Reader("E:/document/学习/UCL/s2/0003/Boundaries Wards/geo_export_6f1ca57a-6a70-4094-b008-70eace180952.shp")
fields = reader.fields[1:]
field_names = [field[0] for field in fields]
buffer = []
for sr in reader.shapeRecords():
    atr = dict(zip(field_names, sr.record))
    geom = sr.shape.__geo_interface__
    buffer.append(dict(type="Feature", \
    geometry=geom, properties=atr)) 
ward_df=geopandas.GeoDataFrame.from_features(buffer)
ward_df = ward_df[["geometry","ward"]]
# ward_df=ward_df.set_crs('epsg:4326')

In [26]:
# print(geopandas.read_file("E:/document/学习/UCL/s2/0003/Boundaries Wards/geo_export_6f1ca57a-6a70-4094-b008-70eace180952.shp").crs) 

epsg:4326


In [3]:
ward_df.head()

,geometry,ward
0,"POLYGON ((-87.69623 41.85755, -87.69625 41.857...",12
1,"POLYGON ((-87.66289 41.79884, -87.66289 41.798...",16
2,"POLYGON ((-87.69818 41.81729, -87.69817 41.817...",15
3,"POLYGON ((-87.65524 41.80883, -87.65524 41.808...",20
4,"POLYGON ((-87.66420 42.02126, -87.66419 42.021...",49


In [3]:
ward_df = ward_df.astype({'ward': 'int'})

In [4]:
crime_ward_raw = pd.read_csv('E:/document/学习/UCL/s2/0003/crime_total.csv')
crime_ward_raw.head()

,ward,ASSAULT_2010,BATTERY_2010,BURGLARY_2010,CRIMINAL DAMAGE_2010,DECEPTIVE PRACTICE_2010,MOTOR VEHICLE THEFT_2010,NARCOTICS_2010,OTHER OFFENSE_2010,ROBBERY_2010,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,1,405,1085,564,921,299,403,425,335,285,...,6662,6262,5363,4655,4553,5259,5366,4837,4809,3411
1,2,186,585,313,561,347,231,160,185,214,...,12678,12340,11407,10864,10676,11349,11456,10088,5266,3245
2,3,636,1878,400,922,263,497,1145,557,373,...,9320,8624,8339,7295,6694,7038,7081,7008,7392,6214
3,4,530,1455,410,765,355,348,737,568,299,...,6045,5396,4916,4612,4603,5047,5176,5562,6961,5443
4,5,514,1494,671,923,322,412,767,630,349,...,8658,8429,7723,6499,6330,6000,6214,6333,5909,5243


In [5]:
crime_ward = pd.merge(crime_ward_raw, ward_df.set_index("ward"), right_index=True,left_on='ward', how='left')
crime_ward = geopandas.GeoDataFrame(crime_ward)
crime_ward.head()

,ward,ASSAULT_2010,BATTERY_2010,BURGLARY_2010,CRIMINAL DAMAGE_2010,DECEPTIVE PRACTICE_2010,MOTOR VEHICLE THEFT_2010,NARCOTICS_2010,OTHER OFFENSE_2010,ROBBERY_2010,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,geometry
0,1,405,1085,564,921,299,403,425,335,285,...,6262,5363,4655,4553,5259,5366,4837,4809,3411,"POLYGON ((-87.67818 41.92863, -87.67817 41.928..."
1,2,186,585,313,561,347,231,160,185,214,...,12340,11407,10864,10676,11349,11456,10088,5266,3245,"POLYGON ((-87.66137 41.92723, -87.66139 41.927..."
2,3,636,1878,400,922,263,497,1145,557,373,...,8624,8339,7295,6694,7038,7081,7008,7392,6214,"POLYGON ((-87.61822 41.85542, -87.61721 41.853..."
3,4,530,1455,410,765,355,348,737,568,299,...,5396,4916,4612,4603,5047,5176,5562,6961,5443,"POLYGON ((-87.62597 41.87336, -87.62597 41.873..."
4,5,514,1494,671,923,322,412,767,630,349,...,8429,7723,6499,6330,6000,6214,6333,5909,5243,"POLYGON ((-87.58337 41.80585, -87.58331 41.805..."


In [6]:
crime_ward.to_file("E:/document/学习/UCL/s2/0003/Chicago_crime_total.json", driver='GeoJSON', encoding="utf-8")